In [ ]:
import pandas as pd
import os
# 기본 경로
base_path = r"C:\Users\henaj\Downloads\278.국내 여행로그 데이터(동부권)\01-1.정식개방데이터\Training\01.원천데이터"
### TASK 1: 활동내역 + 여행지 병합
# 파일 불러오기
activity_df = pd.read_csv(os.path.join(base_path, 'tn_activity_his_활동내역_B.csv'), encoding='utf-8')
traveller_master_df = pd.read_csv(os.path.join(base_path, 'tn_traveller_master_여행객 Master_B.csv'), encoding='utf-8')
travel_df = pd.read_csv(os.path.join(base_path, 'tn_travel_여행_B.csv'), encoding='utf-8')
# Step 1: traveller_master_df에서 TRAVELER_ID, TRAVEL_STATUS_DESTINATION 추출
traveller_dest = traveller_master_df[['TRAVELER_ID', 'TRAVEL_STATUS_DESTINATION']]
# Step 2: travel_df에서 TRAVEL_ID, TRAVELER_ID 추출
travel_id_map = travel_df[['TRAVEL_ID', 'TRAVELER_ID']]
# Step 3: TRAVELER_ID 기준으로 TRAVEL_ID에 TRAVEL_STATUS_DESTINATION 연결
merged_travel = pd.merge(travel_id_map, traveller_dest, on='TRAVELER_ID', how='inner')  # TRAVEL_ID + DEST
# Step 4: 활동내역과 병합 (TRAVEL_ID 기준)
activity_df_trimmed = activity_df[['TRAVEL_ID', 'ACTIVITY_TYPE_CD']]
result1 = pd.merge(activity_df_trimmed, merged_travel[['TRAVEL_ID', 'TRAVEL_STATUS_DESTINATION']], on='TRAVEL_ID', how='inner')
# 결과 저장
result1.to_csv(os.path.join(base_path, 'analysis1.csv'), index=False, encoding='utf-8-sig')
### TASK 2: 숙박소비내역 정제
# 파일 불러오기
lodge_df = pd.read_csv(os.path.join(base_path, 'tn_lodge_consume_his_숙박소비내역_B.csv'), encoding='utf-8')
# 필요한 열만 선택
lodge_df_trimmed = lodge_df[['TRAVEL_ID', 'LODGING_TYPE_CD', 'LOTNO_ADDR']]
# 결측치 제거 (공백, NaN, 'N/A', 'NULL')
lodge_df_cleaned = lodge_df_trimmed.replace(['', ' ', 'N/A', 'NULL'], pd.NA).dropna()
# 결과 저장
lodge_df_cleaned.to_csv(os.path.join(base_path, 'analysis2.csv'), index=False, encoding='utf-8-sig')
print("✅ 모든 작업이 완료되었습니다!")
# 1. 기존 결과 불러오기
analysis1 = pd.read_csv(os.path.join(base_path, 'analysis1.csv'), encoding='utf-8-sig')
analysis2 = pd.read_csv(os.path.join(base_path, 'analysis2.csv'), encoding='utf-8-sig')
# 2. revisit 정보 불러오기
visit_info = pd.read_csv(os.path.join(base_path, 'tn_visit_area_info_방문자정보_B.csv'), encoding='utf-8')
revisit_info = visit_info[['TRAVEL_ID', 'REVISIT_INTENTION']]
# 3. analysis1과 병합
analysis1_updated = pd.merge(analysis1, revisit_info, on='TRAVEL_ID', how='left')
# 4. analysis2와 병합
analysis2_updated = pd.merge(analysis2, revisit_info, on='TRAVEL_ID', how='left')
# 5. 다시 저장
analysis1_updated.to_csv(os.path.join(base_path, 'analysis1.csv'), index=False, encoding='utf-8-sig')
analysis2_updated.to_csv(os.path.join(base_path, 'analysis2.csv'), index=False, encoding='utf-8-sig')
# 1. 기존 결과 불러오기
analysis1 = pd.read_csv(os.path.join(base_path, 'analysis1.csv'), encoding='utf-8-sig')
analysis2 = pd.read_csv(os.path.join(base_path, 'analysis2.csv'), encoding='utf-8-sig')
# 2. 방문자 정보 불러오기
visit_info = pd.read_csv(
    os.path.join(base_path, 'tn_visit_area_info_방문지정보_B.csv'),
    encoding='utf-8-sig',
    low_memory=False  # <-- 경고 방지
)revisit_info = visit_info[['TRAVEL_ID', 'REVISIT_INTENTION']]
# 3. 병합
analysis1_updated = pd.merge(analysis1, revisit_info, on='TRAVEL_ID', how='left')
analysis2_updated = pd.merge(analysis2, revisit_info, on='TRAVEL_ID', how='left')
# 4. 저장
analysis1_updated.to_csv(os.path.join(base_path, 'analysis1.csv'), index=False, encoding='utf-8-sig')
analysis2_updated.to_csv(os.path.join(base_path, 'analysis2.csv'), index=False, encoding='utf-8-sig')
print("🎉 병합 완료! REVISIT_INTENTION 열이 추가되었습니다.")
# CSV 파일 불러오기
analysis_df = pd.read_csv(f'{base_path}\\analysis2.csv')
visit_info_df = pd.read_csv(f'{base_path}\\tn_visit_area_info_방문지정보_B.csv')
# 필요한 열만 선택 (TRAVEL_ID, REVISIT_INTENTION)
visit_info_df = visit_info_df[['TRAVEL_ID', 'REVISIT_INTENTION']]
# 중복 제거 (동일 TRAVEL_ID에 여러 값이 있을 경우 첫 번째만 남김)